# | default_exp 
Convert Epicurious data to Postgresql

In [ ]:
# | hide
# from datetime import datetime
from hashlib import sha256
# from enum import Enum
import json
import pandas as pd
from sqlalchemy import create_engine
# from pydantic import BaseModel
# import vespa
# import requests
from urllib.parse import urlparse

## Notes
For current MySQL database on NAS
- Column translation to Postgres structure
    - 'id' -> 'source_id'
    - 'recipe_title' -> 'title'
    - 'recipe_url' -> 'url'
    - 'recipe_photo' -> 'photo_url'
    - 'description' -> 'description'
    - 'ingredients' -> 'ingredients'
    - 'steps' -> 'steps'
    - 'cuisine' -> 'cuisines'
- Need to add the following columns:
    - 'origin' = 'AllRecipes'
    - 'language' = 'English'
    - 'mealeon_id' = sha256(unique part of the url)
- Need to convert the following columns:
    - 'cuisine': string -> array
    - 'ingredients': string -> array
    - 'steps': string -> array

In [ ]:
# load MySQL credentials from secrets file
mariadb_key_path = '../secrets/pw.json'

with open(mariadb_key_path, 'r') as fo:
    mariadb_key =  json.loads(fo.read())
user = mariadb_key['user']
password = mariadb_key['password']
host = "10.0.0.218"


In [ ]:
# connect to MySQL

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/scraped_recipes?charset=utf8")

with engine.begin() as connection:
    df = pd.read_sql_table(
        table_name='all_recipes_world_cuisines', 
        con=connection, 
        columns=['id', 'recipe_title', 'recipe_url', 'photo', 'description', 'ingredients', 'steps', 'cuisine']
    )

df

,id,recipe_title,recipe_url,photo,description,ingredients,steps,cuisine
0,255989,Spicy Chicken and Hominy Mexican Soup,https://www.allrecipes.com/recipe/255989/spicy...,https://images.media-allrecipes.com/userphotos...,"Colorful, warm and spicy, this soup is destine...","1 tablespoon olive oil||2 chicken breasts, cu...",Heat oil in a large pot over medium-high heat....,Mexican
1,246332,Carnitas - Pressure Cooker,https://www.allrecipes.com/recipe/246332/carni...,https://images.media-allrecipes.com/userphotos...,Savory and tender pork carnitas.,1 (6 pound) pork butt roast||1 ½ tablespoons s...,Trim excess fat from pork butt; cut pork into ...,Mexican
2,246118,Agua Fresca de Pepino (Cucumber Limeade),https://www.allrecipes.com/recipe/246118/agua-...,https://images.media-allrecipes.com/userphotos...,This is another common agua fresca from Mexico...,"5 cups water, or to taste||3 cucumbers, peele...","Blend 2 cups water, cucumbers, lime juice, and...",Mexican
3,244940,Drowned Beef Sandwich with Chipotle Sauce (Tor...,https://www.allrecipes.com/recipe/244940/drown...,https://images.media-allrecipes.com/userphotos...,"Served with a spicy beef broth for dipping, th...",12 ounces chipotle cooking sauce (such a Knorr...,"Combine chipotle cooking sauce, beef broth, an...",Mexican
4,242252,Secret Salsa,https://www.allrecipes.com/recipe/242252/secre...,https://images.media-allrecipes.com/userphotos...,Sweet and spicy pico de gallo-style salsa with...,¼ cup chopped fresh parsley||¼ cup chopped fre...,"Mix parsley, cilantro, lime juice, salt, garli...",Mexican
...,...,...,...,...,...,...,...,...
4776,272454,Curry Orange Swai,https://www.allrecipes.com/recipe/272454/curry...,https://images.media-allrecipes.com/userphotos...,I came up with this tasty swai recipe when I w...,2 (8 ounce) fillets swai fish||2 teaspoons gro...,Lay fish in a glass dish; season both sides wi...,Indian
4777,75514,Fruited Tofu Curry Salad,https://www.allrecipes.com/recipe/75514/fruite...,https://images.media-allrecipes.com/userphotos...,"Vegetarian salad, with a lot of flavor. Serve ...","½ cup white rice||2 cups extra-firm tofu, drai...",In a saucepan bring water to a boil. Add rice ...,Indian
4778,281517,Taste of India Roasted Root Vegetables,https://www.allrecipes.com/recipe/281517/taste...,https://images.media-allrecipes.com/userphotos...,"Though not a traditional Indian dish, since th...","1 medium red onion, cut into 8 wedges||3 carr...",Preheat the oven to 425 degrees F (220 degrees...,Indian
4779,83352,Indian Mustard Fish,https://www.allrecipes.com/recipe/83352/indian...,https://images.media-allrecipes.com/userphotos...,A traditional Bengali (East Indian) dish. It i...,3 tablespoons mustard seed||5 green chile pep...,Place the mustard seed and chile peppers in a ...,Indian


In [ ]:
df.rename(columns={
            'id': 'source_id', 
            'recipe_title':'title', 
            'recipe_url':'url', 
            'photo': 'photo_url', 
            'description': 'description', 
            'ingredients': 'ingredients', 
            'steps': 'steps', 
            'cuisine': 'cuisines'
            }
        , inplace=True
)

df

,source_id,title,url,photo_url,description,ingredients,steps,cuisines
0,255989,Spicy Chicken and Hominy Mexican Soup,https://www.allrecipes.com/recipe/255989/spicy...,https://images.media-allrecipes.com/userphotos...,"Colorful, warm and spicy, this soup is destine...","1 tablespoon olive oil||2 chicken breasts, cu...",Heat oil in a large pot over medium-high heat....,Mexican
1,246332,Carnitas - Pressure Cooker,https://www.allrecipes.com/recipe/246332/carni...,https://images.media-allrecipes.com/userphotos...,Savory and tender pork carnitas.,1 (6 pound) pork butt roast||1 ½ tablespoons s...,Trim excess fat from pork butt; cut pork into ...,Mexican
2,246118,Agua Fresca de Pepino (Cucumber Limeade),https://www.allrecipes.com/recipe/246118/agua-...,https://images.media-allrecipes.com/userphotos...,This is another common agua fresca from Mexico...,"5 cups water, or to taste||3 cucumbers, peele...","Blend 2 cups water, cucumbers, lime juice, and...",Mexican
3,244940,Drowned Beef Sandwich with Chipotle Sauce (Tor...,https://www.allrecipes.com/recipe/244940/drown...,https://images.media-allrecipes.com/userphotos...,"Served with a spicy beef broth for dipping, th...",12 ounces chipotle cooking sauce (such a Knorr...,"Combine chipotle cooking sauce, beef broth, an...",Mexican
4,242252,Secret Salsa,https://www.allrecipes.com/recipe/242252/secre...,https://images.media-allrecipes.com/userphotos...,Sweet and spicy pico de gallo-style salsa with...,¼ cup chopped fresh parsley||¼ cup chopped fre...,"Mix parsley, cilantro, lime juice, salt, garli...",Mexican
...,...,...,...,...,...,...,...,...
4776,272454,Curry Orange Swai,https://www.allrecipes.com/recipe/272454/curry...,https://images.media-allrecipes.com/userphotos...,I came up with this tasty swai recipe when I w...,2 (8 ounce) fillets swai fish||2 teaspoons gro...,Lay fish in a glass dish; season both sides wi...,Indian
4777,75514,Fruited Tofu Curry Salad,https://www.allrecipes.com/recipe/75514/fruite...,https://images.media-allrecipes.com/userphotos...,"Vegetarian salad, with a lot of flavor. Serve ...","½ cup white rice||2 cups extra-firm tofu, drai...",In a saucepan bring water to a boil. Add rice ...,Indian
4778,281517,Taste of India Roasted Root Vegetables,https://www.allrecipes.com/recipe/281517/taste...,https://images.media-allrecipes.com/userphotos...,"Though not a traditional Indian dish, since th...","1 medium red onion, cut into 8 wedges||3 carr...",Preheat the oven to 425 degrees F (220 degrees...,Indian
4779,83352,Indian Mustard Fish,https://www.allrecipes.com/recipe/83352/indian...,https://images.media-allrecipes.com/userphotos...,A traditional Bengali (East Indian) dish. It i...,3 tablespoons mustard seed||5 green chile pep...,Place the mustard seed and chile peppers in a ...,Indian


In [ ]:
df['url'][0]

'https://www.allrecipes.com/recipe/255989/spicy-chicken-and-hominy-mexican-soup/'

In [ ]:
df['url'][0][34:]

for url in df['url'][0:20]:
    print(url[34:])

255989/spicy-chicken-and-hominy-mexican-soup/
246332/carnitas-pressure-cooker/
246118/agua-fresca-de-pepino-cucumber-limeade/
244940/drowned-beef-sandwich-with-chipotle-sauce-torta-ahogada/
242252/secret-salsa/
239051/easy-chicken-flautas/
235756/easy-mexican-sopes/
234881/jalapeno-and-cucumber-margarita/
46653/taco-seasoning-i/
14231/guacamole/
232967/fajita-seasoning/
13351/chicken-tortilla-soup-i/
7399/tres-leches-milk-cake/
281802/creamy-white-chicken-chili-with-salsa-verde/
14064/easy-guacamole/
157642/homemade-flour-tortillas/
20680/easy-mexican-casserole/
27072/mexican-rice-ii/
16678/slow-cooker-taco-soup/
7224/pan-de-muertos-mexican-bread-of-the-dead/


In [ ]:
origin = "AllRecipes"


In [ ]:
def unique_name_from_str(string: str) -> str:
    """
    Generates a unique id name. Preferably, the input string is the netloc and path of the URL
    """
    return sha256(string.encode("utf8")).hexdigest()

In [ ]:
def mealeon_id_assigner(string:str) -> str:
    """
    Generates a unique id name from original URL pushed through the sha256 hash function
    """
    parsed_url = urlparse(string)
    sha_loc_path = unique_name_from_str(f"{parsed_url.netloc}{parsed_url.path}")

    return f"{origin}-{sha_loc_path}"


In [ ]:
df['mealeon_id'] = df['url'].apply(unique_name_from_str)

df

,source_id,title,url,photo_url,description,ingredients,steps,cuisines,mealeon_id
0,255989,Spicy Chicken and Hominy Mexican Soup,https://www.allrecipes.com/recipe/255989/spicy...,https://images.media-allrecipes.com/userphotos...,"Colorful, warm and spicy, this soup is destine...","1 tablespoon olive oil||2 chicken breasts, cu...",Heat oil in a large pot over medium-high heat....,Mexican,5d0905279c7cf0191508accd4079125e6faf346be0f05e...
1,246332,Carnitas - Pressure Cooker,https://www.allrecipes.com/recipe/246332/carni...,https://images.media-allrecipes.com/userphotos...,Savory and tender pork carnitas.,1 (6 pound) pork butt roast||1 ½ tablespoons s...,Trim excess fat from pork butt; cut pork into ...,Mexican,2e0a3f03ae0eb54fee720ba771bb8370b67142def3655b...
2,246118,Agua Fresca de Pepino (Cucumber Limeade),https://www.allrecipes.com/recipe/246118/agua-...,https://images.media-allrecipes.com/userphotos...,This is another common agua fresca from Mexico...,"5 cups water, or to taste||3 cucumbers, peele...","Blend 2 cups water, cucumbers, lime juice, and...",Mexican,12e7ce6be42ecff0c5dee55413f93d9fdb02c19fa0f9da...
3,244940,Drowned Beef Sandwich with Chipotle Sauce (Tor...,https://www.allrecipes.com/recipe/244940/drown...,https://images.media-allrecipes.com/userphotos...,"Served with a spicy beef broth for dipping, th...",12 ounces chipotle cooking sauce (such a Knorr...,"Combine chipotle cooking sauce, beef broth, an...",Mexican,f8e2a625d392dd1eb9b16d5fd0b9feb3b95b51e8fb2645...
4,242252,Secret Salsa,https://www.allrecipes.com/recipe/242252/secre...,https://images.media-allrecipes.com/userphotos...,Sweet and spicy pico de gallo-style salsa with...,¼ cup chopped fresh parsley||¼ cup chopped fre...,"Mix parsley, cilantro, lime juice, salt, garli...",Mexican,982de8ddcf0333c5e733552e1ef233b17ca4114ab2ecc8...
...,...,...,...,...,...,...,...,...,...
4776,272454,Curry Orange Swai,https://www.allrecipes.com/recipe/272454/curry...,https://images.media-allrecipes.com/userphotos...,I came up with this tasty swai recipe when I w...,2 (8 ounce) fillets swai fish||2 teaspoons gro...,Lay fish in a glass dish; season both sides wi...,Indian,58a9388cbdcac377bdeb70cce09be7101a59e9608a6ab0...
4777,75514,Fruited Tofu Curry Salad,https://www.allrecipes.com/recipe/75514/fruite...,https://images.media-allrecipes.com/userphotos...,"Vegetarian salad, with a lot of flavor. Serve ...","½ cup white rice||2 cups extra-firm tofu, drai...",In a saucepan bring water to a boil. Add rice ...,Indian,40143586dab73da010dd22cbfb66b164ce68ba9f37e2e6...
4778,281517,Taste of India Roasted Root Vegetables,https://www.allrecipes.com/recipe/281517/taste...,https://images.media-allrecipes.com/userphotos...,"Though not a traditional Indian dish, since th...","1 medium red onion, cut into 8 wedges||3 carr...",Preheat the oven to 425 degrees F (220 degrees...,Indian,77c84f71a592a1de94164412e3f569758c239f8c82395b...
4779,83352,Indian Mustard Fish,https://www.allrecipes.com/recipe/83352/indian...,https://images.media-allrecipes.com/userphotos...,A traditional Bengali (East Indian) dish. It i...,3 tablespoons mustard seed||5 green chile pep...,Place the mustard seed and chile peppers in a ...,Indian,8def2fb0afdeb8322fcc7c442e0f7d60ad192f5ba853d9...


In [ ]:
# | hide
nbdev.nbdev_export()